In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/train.csv")
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#Dropping NAN value
df=df.dropna()
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
#Get the independent features

X=df.drop('label',axis=1)

In [ ]:
#Get the dependent features
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.5.0


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
#Vocabulary Size
voc_size=5000

In [ ]:
#One Hot representation
messages=X.copy()

In [ ]:
messages.reset_index(inplace=True )

In [ ]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]'," ",messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words("english")]
  review=" ".join(review)
  corpus.append(review)

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[731, 1178, 2800, 73, 3813, 4831, 2780, 3726, 4712, 3480],
 [1482, 4733, 634, 4790, 809, 1408, 2389],
 [3355, 4469, 3809, 2401],
 [4633, 77, 2361, 1275, 1327, 2525],
 [4759, 809, 520, 2815, 2946, 2981, 809, 2982, 4891, 3488],
 [2568,
  2575,
  4295,
  371,
  3181,
  351,
  1438,
  3447,
  974,
  2112,
  2167,
  2046,
  333,
  4436,
  2389],
 [3726, 4593, 3548, 1422, 35, 4541, 4822, 1434, 4083, 3806, 4336],
 [2715, 2229, 2536, 3974, 1322, 4810, 351, 4021, 4083, 3806, 4336],
 [3480, 3948, 2432, 2259, 4453, 1140, 742, 2459, 351, 2070],
 [2360, 3464, 1096, 2442, 2365, 1101, 2814, 3420],
 [1731, 3077, 4873, 1991, 4589, 4604, 505, 1112, 723, 31, 3854],
 [1275, 960, 3813, 1140, 351, 1322],
 [4909, 418, 3031, 3573, 4187, 2106, 2699, 4569, 2255],
 [2178, 2752, 2180, 2210, 163, 356, 3647, 4083, 3806, 4336],
 [3818, 1058, 4369, 4930, 3299, 4083, 3806, 4336],
 [941, 2714, 4052, 444, 4387, 3788, 3750, 1536, 4095, 1043],
 [1614, 3432, 4733],
 [700, 3896, 1492, 3014, 351, 3711, 704, 2389],
 [1287, 3

In [ ]:
#Embedding Representation
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding="pre",maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3726 4712 3480]
 [   0    0    0 ...  809 1408 2389]
 [   0    0    0 ... 4469 3809 2401]
 ...
 [   0    0    0 ... 4083 3806 4336]
 [   0    0    0 ...  401 2376 3118]
 [   0    0    0 ...  383 3534 1068]]


In [ ]:
#Creating model
embedding_vector_feature=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.2,random_state=42)

In [ ]:
#Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 10s 33ms/step - loss: 0.3033 - accuracy: 0.8628 - val_loss: 0.2146 - val_accuracy: 0.9133
Epoch 2/10
229/229 [==============================] - 7s 31ms/step - loss: 0.1365 - accuracy: 0.9463 - val_loss: 0.1964 - val_accuracy: 0.9152
Epoch 3/10
229/229 [==============================] - 7s 31ms/step - loss: 0.1007 - accuracy: 0.9617 - val_loss: 0.2244 - val_accuracy: 0.9106
Epoch 4/10
229/229 [==============================] - 7s 31ms/step - loss: 0.0773 - accuracy: 0.9713 - val_loss: 0.2504 - val_accuracy: 0.9106
Epoch 5/10
229/229 [==============================] - 7s 31ms/step - loss: 0.0569 - accuracy: 0.9796 - val_loss: 0.3103 - val_accuracy: 0.9065
Epoch 6/10
229/229 [==============================] - 7s 31ms/step - loss: 0.0448 - accuracy: 0.9858 - val_loss: 0.3492 - val_accuracy: 0.9092
Epoch 7/10
229/229 [==============================] - 7s 31ms/step - loss: 0.0297 - accuracy: 0.9909 - val_loss: 0.3810 - val_accuracy: 0.907

In [ ]:
#Performance Metrics and Accuracy
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1882,  200],
       [ 139, 1436]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9073010664479081

In [ ]:
#Adding Dropout Layer
from tensorflow.keras.layers import Dropout
embedding_vector_feature=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation="sigmoid"))
model1.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
print(model1.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 9s 33ms/step - loss: 0.3111 - accuracy: 0.8527 - val_loss: 0.1951 - val_accuracy: 0.9130
Epoch 2/10
229/229 [==============================] - 7s 31ms/step - loss: 0.1409 - accuracy: 0.9438 - val_loss: 0.2012 - val_accuracy: 0.9125
Epoch 3/10
229/229 [==============================] - 7s 31ms/step - loss: 0.1091 - accuracy: 0.9594 - val_loss: 0.2299 - val_accuracy: 0.9128
Epoch 4/10
229/229 [==============================] - 7s 31ms/step - loss: 0.0907 - accuracy: 0.9658 - val_loss: 0.2272 - val_accuracy: 0.9130
Epoch 5/10
229/229 [==============================] - 7s 31ms/step - loss: 0.0736 - accuracy: 0.9714 - val_loss: 0.2569 - val_accuracy: 0.9117
Epoch 6/10
229/229 [==============================] - 7s 31ms/step - loss: 0.0621 - accuracy: 0.9778 - val_loss: 0.2781 - val_accuracy: 0.9016
Epoch 7/10
229/229 [==============================] - 7s 31ms/step - loss: 0.0491 - accuracy: 0.9827 - val_loss: 0.3337 - val_accuracy: 0.9098

In [ ]:
y_pred1=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[1600,  482],
       [1022,  553]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.5887339349193328